get the files

In [ ]:
#download the train, test and validate sets
!wget https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l&dl=0 # train
!wget https://www.dropbox.com/scl/fi/toycvi2k3h0dz42gl3vk0/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7&dl=0 # test
!wget https://www.dropbox.com/scl/fi/khtr33xeykwpjjezlbkdd/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz&dl=0 # validation

--2024-03-13 09:43:58--  https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucca742609f0c42ed638ba2d7865.dl.dropboxusercontent.com/cd/0/inline/CPBYGgaOLDke6-whkUei06EkTmHRO8mv0voZsKCXiVp-DXMKOXcHadzr6IWEn6NMZWCtQdusYBmRzkTO8lAcRbVLdEPL4B0ZLTl8PY42AQut8mNQLBB4BbA42opOmdwFBiSKk22MfHYr1CTWsG-vQa8e/file# [following]
--2024-03-13 09:43:58--  https://ucca742609f0c42ed638ba2d7865.dl.dropboxusercontent.com/cd/0/inline/CPBYGgaOLDke6-whkUei06EkTmHRO8mv0voZsKCXiVp-DXMKOXcHadzr6IWEn6NMZWCtQdusYBmRzkTO8lAcRbVLdEPL4B0ZLTl8PY42AQut8mNQLBB4BbA42opOmdwFBiSKk22MfHYr1CTWsG-vQa8e/file
Resolving ucca742609f0c42ed638ba2d7865.dl.dropboxusercontent.com (ucca742609f0c42ed638ba2d7865.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:

In [ ]:
# unzip the downloaded zip files
!unzip /content/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l # train
!unzip /content/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7 # test
!unzip /content/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz # validation

make the model

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_dir = '/content/train'
test_dir = '/content/test'
valid_dir = '/content/validation'

size=(224,224)

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=size,
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(valid_dir,
                                                        target_size=size,
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator))

calculate the test set accuracy metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [ ]:
# load the test set
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = np.argmax(predictions, axis=1)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

In [ ]:
# plot roc curve
plt.plot(fpr[0], tpr[0], label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
model.save('disaster_vgg16_1.h5')